In [1]:
def add_hello(func):
  def wrapper(*args, **kwargs):
    print('Hello')
    return func(*args, **kwargs)
  return wrapper

# Decorate print_sum() with the add_hello() decorator
@add_hello
def print_sum(a, b):
  """Adds two numbers and prints the sum"""
  print(a + b)

In [2]:
print_sum(10, 20)
# Define the docstring
print_sum_docstring = print_sum.__doc__
print(print_sum_docstring)

Hello
30
None


In [3]:
from functools import wraps

def add_hello(func):
  # Decorate wrapper() so that it keeps func()'s metadata
  @wraps(func)
  def wrapper(*args, **kwargs):
    """Print 'hello' and then call the decorated function."""
    print('Hello')
    return func(*args, **kwargs)
  return wrapper
  
@add_hello
def print_sum(a, b):
  """Adds two numbers and prints the sum"""
  print(a + b)
  
print_sum(10, 20)
print_sum_docstring = print_sum.__doc__
print(print_sum_docstring)

Hello
30
Adds two numbers and prints the sum


In [4]:
def memoize(func):
    """
    Store the results of the decorated function for fast lookup
    """
    # Store results in a dict that maps arguments to results
    cache = {}

    # Define the wrapper function to return.
    def wrapper(*args, **kwargs):
        
        kwargs_key = tuple(sorted(kwargs.items()))
        print("Now printing kwargs_key: ", kwargs_key)

        if (args, kwargs_key) not in cache:
            print(f"These values {(args, kwargs_key)} are not there in the cache", )
            cache[(args, kwargs_key)] = func(*args, **kwargs)
        return cache[(args, kwargs_key)]

    return wrapper


In [5]:
@memoize
def slow_function(a, b):
    return a + b

for i, cell in enumerate(slow_function.__closure__):
    print(f"Closure cell {i}: {cell.cell_contents}")


Closure cell 0: {}
Closure cell 1: <function slow_function at 0x1122ec720>


What variables from the outer scope are used inside wrapper?

    func: used to call the original function

    cache: used to store results

Therefore:

    Both func and cache are free variables inside wrapper.

    So both will be stored in wrapper.__closure__

In [6]:
print(slow_function.__code__.co_freevars)
# Output: ('func', 'cache')

('cache', 'func')


Yes, __closure__[i].cell_contents can contain the original function, if it's referenced inside the inner function.

In your case, it does — because func is used inside wrapper.


⚠️ Final Caveat

    Not all decorators behave the same way.

If the decorator doesn't use the original function directly in the inner wrapper (e.g., uses functools.partial, or returns another wrapper-of-a-wrapper), the closure may or may not contain the original function.

That's why functools.wraps is safer — it explicitly assigns ```__wrapped__```, while closures depend on how things are written.
